In [23]:
pip install us

Note: you may need to restart the kernel to use updated packages.


In [78]:
import numpy as np
import pandas as pd
import us as us

import io
from zipfile import ZipFile

In [62]:
# raw = pd.read_csv('data/cps_00001.csv')

# raw_2 = pd.read_csv('http://localhost:8888/edit/UBICenter/min-wage/jb/data/cps_00002.csv.gz')

raw_2 = pd.read_csv('UBICenter/min-wage/jb/data/cps_00002.csv.gz')


Only include 2018-2020 ASEC, then monthly data for 2020. Need year and month, hourwage, state and county (need that), diffany (disability), citizenship, spm_family_unit, spm resources, spm pov threshold

In [63]:
raw_2

,YEAR,SERIAL,MONTH,HWTFINL,CPSID,ASECFLAG,ASECWTH,STATEFIP,COUNTY,PERNUM,...,CITIZEN,DIFFANY,SPMPOV,SPMTOTRES,SPMNADULTS,SPMNCHILD,SPMNPERS,SPMTHRESH,SPMFAMUNIT,HOURWAGE
0,2018,2,3,NaN,20171202892400,1.0,1316.02,23,0,1,...,1,2,0.0,47514.000,1.0,0.0,1.0,11004.31484,2001.0,999.99
1,2018,4,3,NaN,20180302858200,1.0,824.29,23,0,1,...,1,1,0.0,105232.008,2.0,3.0,5.0,26449.08415,4001.0,999.99
2,2018,4,3,NaN,20180302858200,1.0,824.29,23,0,2,...,1,1,0.0,105232.008,2.0,3.0,5.0,26449.08415,4001.0,999.99
3,2018,4,3,NaN,20180302858200,1.0,824.29,23,0,3,...,1,0,0.0,105232.008,2.0,3.0,5.0,26449.08415,4001.0,999.99
4,2018,4,3,NaN,20180302858200,1.0,824.29,23,0,4,...,1,0,0.0,105232.008,2.0,3.0,5.0,26449.08415,4001.0,999.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027643,2020,68927,12,438.2519,20201106884600,NaN,NaN,56,0,2,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.99
2027644,2020,68931,12,334.2817,20191207029100,NaN,NaN,56,0,1,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.99
2027645,2020,68931,12,334.2817,20191207029100,NaN,NaN,56,0,2,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.99
2027646,2020,68931,12,334.2817,20191207029100,NaN,NaN,56,0,3,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.99


In [20]:
raw_2.HOURWAGE.describe()

count    2.027648e+06
mean     9.466096e+02
std      2.224880e+02
min      4.000000e-01
25%      9.999900e+02
50%      9.999900e+02
75%      9.999900e+02
max      9.999900e+02
Name: HOURWAGE, dtype: float64

In [39]:
min_wage_data = raw_2[raw_2.HOURWAGE < 999.99]

# check IPUMS field documentation
# make sure to copy dataset before starting

min_wage_data.HOURWAGE.describe()

count    110380.000000
mean         19.408039
std          10.921556
min           0.400000
25%          12.500000
50%          16.000000
75%          23.000000
max          99.990000
Name: HOURWAGE, dtype: float64

In [40]:
min_wage_data['under_15'] = min_wage_data.HOURWAGE < 15

min_wage_data.under_15

<ipython-input-40-93266fefc1c0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_wage_data['under_15'] = min_wage_data.HOURWAGE < 15


38         False
71          True
88          True
220         True
221        False
           ...  
2027559    False
2027560     True
2027616    False
2027631    False
2027632    False
Name: under_15, Length: 110380, dtype: bool

In [41]:
min_wage_data

,YEAR,SERIAL,MONTH,HWTFINL,CPSID,ASECFLAG,ASECWTH,STATEFIP,COUNTY,PERNUM,...,DIFFANY,SPMPOV,SPMTOTRES,SPMNADULTS,SPMNCHILD,SPMNPERS,SPMTHRESH,SPMFAMUNIT,HOURWAGE,under_15
38,2018,26,3,NaN,20171202868100,1.0,1120.98,23,0,2,...,1,0.0,60061.008,2.0,0.0,2.0,13632.36586,26001.0,16.00,False
71,2018,40,3,NaN,20171202867300,1.0,716.67,23,0,1,...,1,1.0,399.232,1.0,3.0,4.0,22563.73248,40001.0,9.00,True
88,2018,46,3,NaN,20161202950000,1.0,1153.61,23,0,1,...,1,0.0,19710.008,1.0,0.0,1.0,10974.89364,46001.0,7.50,True
220,2018,143,3,NaN,20171202879300,1.0,1266.70,23,0,1,...,1,0.0,54334.016,2.0,0.0,2.0,15474.60003,143001.0,12.50,True
221,2018,143,3,NaN,20171202879300,1.0,1266.70,23,0,2,...,1,0.0,54334.016,2.0,0.0,2.0,15474.60003,143001.0,50.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027559,2020,68876,12,369.0110,20190906988000,NaN,NaN,56,0,2,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.00,False
2027560,2020,68876,12,369.0110,20190906988000,NaN,NaN,56,0,3,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.00,True
2027616,2020,68904,12,381.7554,20190906991200,NaN,NaN,56,0,2,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.00,False
2027631,2020,68920,12,254.8724,20200906860600,NaN,NaN,56,0,1,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.25,False


In [43]:
min_wage_data['u15_t'] = np.where(min_wage_data.under_15 == True, 1, 0)

min_wage_data['u15_f'] = np.where(min_wage_data.under_15 == False, 1, 0)

min_wage_data




<ipython-input-43-543a574579f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_wage_data['u15_t'] = np.where(min_wage_data.under_15 == True, 1, 0)
<ipython-input-43-543a574579f4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_wage_data['u15_f'] = np.where(min_wage_data.under_15 == False, 1, 0)


,YEAR,SERIAL,MONTH,HWTFINL,CPSID,ASECFLAG,ASECWTH,STATEFIP,COUNTY,PERNUM,...,SPMTOTRES,SPMNADULTS,SPMNCHILD,SPMNPERS,SPMTHRESH,SPMFAMUNIT,HOURWAGE,under_15,u15_t,u15_f
38,2018,26,3,NaN,20171202868100,1.0,1120.98,23,0,2,...,60061.008,2.0,0.0,2.0,13632.36586,26001.0,16.00,False,0,1
71,2018,40,3,NaN,20171202867300,1.0,716.67,23,0,1,...,399.232,1.0,3.0,4.0,22563.73248,40001.0,9.00,True,1,0
88,2018,46,3,NaN,20161202950000,1.0,1153.61,23,0,1,...,19710.008,1.0,0.0,1.0,10974.89364,46001.0,7.50,True,1,0
220,2018,143,3,NaN,20171202879300,1.0,1266.70,23,0,1,...,54334.016,2.0,0.0,2.0,15474.60003,143001.0,12.50,True,1,0
221,2018,143,3,NaN,20171202879300,1.0,1266.70,23,0,2,...,54334.016,2.0,0.0,2.0,15474.60003,143001.0,50.00,False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027559,2020,68876,12,369.0110,20190906988000,NaN,NaN,56,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,21.00,False,0,1
2027560,2020,68876,12,369.0110,20190906988000,NaN,NaN,56,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,12.00,True,1,0
2027616,2020,68904,12,381.7554,20190906991200,NaN,NaN,56,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,35.00,False,0,1
2027631,2020,68920,12,254.8724,20200906860600,NaN,NaN,56,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,32.25,False,0,1


In [45]:
mw_gby_state = min_wage_data.groupby(['STATEFIP']).sum()

# not good
mw_gby_state

,YEAR,SERIAL,MONTH,HWTFINL,CPSID,ASECFLAG,ASECWTH,COUNTY,PERNUM,WTFINL,...,SPMTOTRES,SPMNADULTS,SPMNCHILD,SPMNPERS,SPMTHRESH,SPMFAMUNIT,HOURWAGE,under_15,u15_t,u15_f
STATEFIP,,,,,,,,,,,,,,,,,,,,,
1,4598844,22923640,15251,3.864837e+06,45971319265537128,660.0,614665.04,447078,4036,4.065437e+06,...,2.530102e+07,832.0,254.0,1086.0,7.640559e+06,2.154109e+10,41802.98,988,988,1289
2,2799266,26979611,8902,5.652240e+05,27982598264051900,452.0,99004.23,0,2651,5.976224e+05,...,2.085901e+07,628.0,293.0,921.0,7.654444e+06,2.489512e+10,30961.18,411,411,975
4,3914129,31661298,12564,6.161578e+06,39126942369734592,580.0,955857.30,7104059,3804,6.364609e+06,...,2.551791e+07,828.0,323.0,1151.0,8.666531e+06,2.707315e+10,37300.20,751,751,1187
5,3922160,27700525,12372,2.374110e+06,39206613105204584,574.0,354256.61,0,3408,2.434826e+06,...,2.360299e+07,818.0,278.0,1096.0,7.404150e+06,2.122876e+10,34326.19,870,870,1072
6,18374865,205389456,58369,2.965791e+07,183678010143732928,2873.0,4033188.86,43035156,18796,3.099417e+07,...,1.303442e+08,4728.0,1433.0,6161.0,5.644077e+07,1.521682e+11,195505.16,2980,2980,6118
8,2764908,29881913,9208,4.666584e+06,27639515497134000,411.0,790652.85,4396314,2577,4.742234e+06,...,1.904478e+07,550.0,170.0,720.0,6.308138e+06,1.869451e+10,27917.38,428,428,941
9,2344831,11515393,7393,2.878239e+06,23438830475669200,412.0,516457.21,5718631,2209,3.037691e+06,...,1.809062e+07,587.0,159.0,746.0,6.650098e+06,1.599529e+09,24549.69,380,380,781
10,2447837,19398700,7681,7.789461e+05,24468624023337400,410.0,147657.89,12123634,2395,8.437445e+05,...,2.074009e+07,665.0,134.0,799.0,6.401383e+06,8.294192e+09,23192.61,450,450,762
11,1698470,15243154,5417,2.812891e+05,16978292494250300,297.0,47015.18,9251841,1462,2.910076e+05,...,1.312684e+07,391.0,109.0,500.0,5.488365e+06,6.991122e+09,19487.20,233,233,608


In [48]:
mw_gby_state['num_u15'] = mw_gby_state.u15_t * mw_gby_state.WTFINL

mw_gby_state['num_u15_f'] = mw_gby_state.u15_f * mw_gby_state.WTFINL

mw_gby_state['pct_below15'] = mw_gby_state.num_u15/(mw_gby_state.num_u15 + mw_gby_state.num_u15_f)

mw_gby_state

,YEAR,SERIAL,MONTH,HWTFINL,CPSID,ASECFLAG,ASECWTH,COUNTY,PERNUM,WTFINL,...,SPMNPERS,SPMTHRESH,SPMFAMUNIT,HOURWAGE,under_15,u15_t,u15_f,num_u15,num_u15_f,pct_below15
STATEFIP,,,,,,,,,,,,,,,,,,,,,
1,4598844,22923640,15251,3.864837e+06,45971319265537128,660.0,614665.04,447078,4036,4.065437e+06,...,1086.0,7.640559e+06,2.154109e+10,41802.98,988,988,1289,4.016651e+09,5.240348e+09,0.433904
2,2799266,26979611,8902,5.652240e+05,27982598264051900,452.0,99004.23,0,2651,5.976224e+05,...,921.0,7.654444e+06,2.489512e+10,30961.18,411,411,975,2.456228e+08,5.826818e+08,0.296537
4,3914129,31661298,12564,6.161578e+06,39126942369734592,580.0,955857.30,7104059,3804,6.364609e+06,...,1151.0,8.666531e+06,2.707315e+10,37300.20,751,751,1187,4.779821e+09,7.554790e+09,0.387513
5,3922160,27700525,12372,2.374110e+06,39206613105204584,574.0,354256.61,0,3408,2.434826e+06,...,1096.0,7.404150e+06,2.122876e+10,34326.19,870,870,1072,2.118299e+09,2.610134e+09,0.447992
6,18374865,205389456,58369,2.965791e+07,183678010143732928,2873.0,4033188.86,43035156,18796,3.099417e+07,...,6161.0,5.644077e+07,1.521682e+11,195505.16,2980,2980,6118,9.236264e+10,1.896224e+11,0.327545
8,2764908,29881913,9208,4.666584e+06,27639515497134000,411.0,790652.85,4396314,2577,4.742234e+06,...,720.0,6.308138e+06,1.869451e+10,27917.38,428,428,941,2.029676e+09,4.462442e+09,0.312637
9,2344831,11515393,7393,2.878239e+06,23438830475669200,412.0,516457.21,5718631,2209,3.037691e+06,...,746.0,6.650098e+06,1.599529e+09,24549.69,380,380,781,1.154322e+09,2.372436e+09,0.327304
10,2447837,19398700,7681,7.789461e+05,24468624023337400,410.0,147657.89,12123634,2395,8.437445e+05,...,799.0,6.401383e+06,8.294192e+09,23192.61,450,450,762,3.796850e+08,6.429333e+08,0.371287
11,1698470,15243154,5417,2.812891e+05,16978292494250300,297.0,47015.18,9251841,1462,2.910076e+05,...,500.0,5.488365e+06,6.991122e+09,19487.20,233,233,608,6.780477e+07,1.769326e+08,0.277051


In [77]:
raw_3 = pd.read_csv('UBICenter/min-wage/jb/data/updated_ds.csv.gz')

# https://github.com/cotrag/min-wage/blob/main/jb/data/cps_00003.csv.gz

raw_3

,YEAR,SERIAL,MONTH,CPSID,ASECFLAG,ASECWTH,STATEFIP,COUNTY,PERNUM,CPSIDP,...,CITIZEN,DIFFANY,POVERTY,SPMWT,SPMTOTRES,SPMNPERS,SPMTHRESH,SPMFAMUNIT,HOURWAGE,QHOURWAG
0,2018,2,3,20171202892400,1,1316.02,23,0,1,20171202892401,...,1,2,23,1316.02,47514.000,1,11004.31484,2001,999.99,0
1,2018,4,3,20180302858200,1,824.29,23,0,1,20180302858201,...,1,1,23,824.29,105232.008,5,26449.08415,4001,999.99,0
2,2018,4,3,20180302858200,1,824.29,23,0,2,20180302858202,...,1,1,23,824.29,105232.008,5,26449.08415,4001,999.99,0
3,2018,4,3,20180302858200,1,824.29,23,0,3,20180302858205,...,1,0,23,824.29,105232.008,5,26449.08415,4001,999.99,0
4,2018,4,3,20180302858200,1,824.29,23,0,4,20180302858204,...,1,0,23,824.29,105232.008,5,26449.08415,4001,999.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518139,2020,91498,3,0,1,444.18,15,15003,1,0,...,1,1,23,444.18,42819.000,1,14770.00000,91498001,999.99,0
518140,2020,91498,3,0,1,444.18,15,15003,2,0,...,3,1,10,388.29,-3574.000,1,14770.00000,91498003,999.99,0
518141,2020,91499,3,0,1,494.05,15,15003,1,0,...,4,2,21,494.05,20868.000,1,17800.00000,91499001,999.99,0
518142,2020,91500,3,0,1,423.84,15,15003,1,0,...,1,1,23,423.84,77447.000,2,27260.00000,91500001,999.99,0


In [80]:
def read_zip(zip_fn, extract_fn=None):
    zf = ZipFile(zip_fn)
    if extract_fn:
        return zf.read(extract_fn)
    else:
        return {name:zf.read(name) for name in zf.namelist()}


bls_data = pd.read_excel(io.BytesIO(read_zip('UBICenter/min-wage/jb/data/oes_2019.xlsx.zip')))

bls_data

 # UBICenter/min-wage/jb/data/updated_ds.csv.gz

TypeError: a bytes-like object is required, not 'dict'

In [81]:
!ls

4160 LAB 1 FINAL.R                   Pictures
4160 LAB WORK??.R                    Poli4781_Tragesser_LabAssignment_1.R
4160 Lab 1.R                         Practice 2.ipynb
4160 Lab 3.R                         Practice 3.ipynb
4160 Lab Assignment 2.R              Practice.ipynb
4160 REAL LAB.R                      Public
4160 lab 2.R                         Tragesser, Chandrakumar, Ray.R
4160.Lab.4.Real.R                    UBICenter
4782_PS2.R                           Untitled.ipynb
Applications                         Untitled1.ipynb
Boxplot Assigment 3                  Untitled2.ipynb
Chandrakumar, Ray, Tragesser         WAIP CV.docx
Creative Cloud Files                 WAIP CV.pdf
Desktop                              bitbuckettutorial
Documents                            cotragesser.Rproj
Downloads                            gd_2
EconHW3.docx                         geyserplot.png
EconHW4.docx                         hello-world
Library                              lab_8.html
Mov